# Build Regression Model based on data_per_zipcode

As seen during our collinearity analysis it seems that zipcode might affect
the property price.


# Build Regression Model based on data_per_zipcode to predict property price

As seen during our collinearity analysis it seems that zipcode might affect
the property price.

So the question is How does that work ?.

Table of Contents
1. [EDA dataset per zipcode](#EDA-dataset-per-zipcode)
4. [Scale and normalise variables](#Scale-and-normalise-variables)
5. [Building the model](#Building-the-model)
6. [Conclusion](#Conclusion)


# Prep-work


In [ ]:
# Import useful librairies and set auto reload
import pandas as pd
import numpy as np
import housing_data as hd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2



##EDA dataset per zipcode


In [ ]:
# Load dataset
data = hd.load_housing_data(with_cat_columns=False)
data.head()


In [ ]:
# Plot property count per zipcode
g = sns.factorplot("zipcode", data=data, aspect=1.5, kind="count", color="b")
g.set_xticklabels(rotation=30)


It looks like we got enough data per zipcode.


**Display price against sqft_living**


In [ ]:
# Split dataset in two group of zipcode for display purpose.
zipcodes = data['zipcode'].unique()
zipcodes.sort()
zipcodes_median = zipcodes[(len(zipcodes) // 2): (len(zipcodes) // 2) + 1][0]

data_set1 = data.loc[data['zipcode'] <= zipcodes_median]
data_set2 = data.loc[data['zipcode'] > zipcodes_median]


In [ ]:
# Display price vs sqft_living on the first subset
plt.figure(figsize=(10,10))
g = sns.FacetGrid(data_set2, col="zipcode", col_wrap=3)
g.map(plt.scatter, "sqft_living", "price")
g.set(xlim=(0, 6000), ylim=(0, 3000000))
g.set_xlabels('')
g.set_ylabels('Property price')


**Conclusion** : The plots seem to be linear. Let's validate this hypothesis
by building a model on a specific zipcode.


## Scale and normalise variables


In [ ]:
# Scale Variables data
log_sqft_living = np.log(data['sqft_living'])
scaled_sqft_living = (log_sqft_living-min(log_sqft_living))/(max(log_sqft_living)-min(log_sqft_living))

data_fin = pd.DataFrame([])
data_fin['sqft_living'] = scaled_sqft_living

scaled_data = data.drop(['sqft_living'], axis=1)
scaled_data = pd.concat([scaled_data, data_fin], axis=1)


# Building the model
We are now going to try to run a simple Regression against our dataset


In [ ]:
# Get sample data from specific zipcode
zipcode = 98072
data_per_zipcode = scaled_data.loc[scaled_data['zipcode'] == zipcode]


In [ ]:
# Build formula
# Notes that we are especting a corrolation between sqft_living and price_sqft
formula = 'price ~ sqft_living'


In [ ]:
# Run simple prediction
model = ols(formula=formula, data=data_per_zipcode).fit()
model.summary()



**Observations** : The Adj. R-squared is pretty low and our variables coef
p-values are low. This doesn't look good.


# Conclusion
We saw that the model built based on zipcode proximity is not really accurate
We need to come up with a more precise model.